## ADD: LOOP FOR "FOLDER IN FOLDER" Logic

In [16]:
from ast import literal_eval
from pyspark.sql.functions import *

dfheader = spark.read.format("csv").option("header","true").load("Files/Logs/Header_File.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Logs/Header_File.csv".
dfheader = dfheader.limit(0)

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 18, Finished, Available, Finished)

In [17]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/Logs/Metafour - Storage Account to OneLake/213aac1c-47ba-428c-90cf-3fb1346725e1/91972b26-bf78-82cf-a3db-8706e09c99b2_0.txt"
df =  pd.read_csv("/lakehouse/default/Files/Logs/Metafour - Storage Account to OneLake/213aac1c-47ba-428c-90cf-3fb1346725e1/91972b26-bf78-82cf-a3db-8706e09c99b2_0.txt", sep=',')
#display(df)

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 19, Finished, Available, Finished)

In [18]:
dfSplit = pd.DataFrame(df['OperationItem'].tolist(), index=df.index).add_prefix('OperationItem_')

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 20, Finished, Available, Finished)

In [19]:
symbol_list = list((" ",",",";","{","}","(",")","\n","\t","=","\r\n","[","]"))
symbol_list

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 21, Finished, Available, Finished)

[' ', ',', ';', '{', '}', '(', ')', '\n', '\t', '=', '\r\n', '[', ']']

In [20]:
dfSplit

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 22, Finished, Available, Finished)

,OperationItem_0
0,"""""""Wesley McCurry"""",""""27/08/2024"""",""""08:30"""",""..."


In [21]:
pddf = []

for i in dfSplit.values:
    pddf.insert(0,i)
print(pddf)

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 23, Finished, Available, Finished)

[array(['"""Wesley McCurry"",""27/08/2024"",""08:30"",""HN002"",""HealthNet Swadlincote"",""SH1505571"","""","""","""",""Unit 1  Orbit Business Park"",""Alfred Eley Close"","""",""SWADLINCOTE"",""Swadlincote"",""HealthNet Homecare (Swadlincote)"",""GB"",""27/08/2024"",""United Kingdom"",""DE11 0WU"",""Chilled"","""","""",""Y"",""Ellen Mary O""Neill"",""28/08/2024"","""",""33 THE PINES AVENUE"","""","""",""COLERAINE"",""COLERAINE"",""33 THE PINES AVENUE"",""Ellen Mary O\'Neill"",""United Kingdom"",""GB"","""",""07845779950"",""BT52 3FR"",""28/08/2024"",""14:26"",""N"","""","""","""","""",""Delivered"",""Delivered"",""28/08/2024"",""15:10"",""14:26:00"",""302 Belfast"",""ALL DAY"",""13:26-15:26"",""1"",""10.00"",""100.00"",""SH1505571"",""0.00"",""20:00"",""""\r\n",'],
      dtype=object)]


In [55]:
data = str(pddf)
dfidea = pd.DataFrame([x.split(',') for x in data.split('\n')])
display(dfidea)

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 57, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 07722f7f-b316-461d-b31d-7e10af0b5afa)

In [84]:
dfSplitSparkDf = spark.createDataFrame(dfidea, dfheader.schema)
dfSplitSparkDf = dfSplitSparkDf.limit(1)

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 86, Finished, Available, Finished)

In [87]:
regexp_patterns = ('""', '"', "'", 'array', '\\r\\n')
non_temporal_regexp_patterns = ('""', '"', "'", '\\\\[a-z0-9\s]+\\\\[a-z0-9\s]+', '[^a-zA-Z0-9\s]', 'array', '\\r\\n')

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 89, Finished, Available, Finished)

In [88]:
#column_list = dfSplitSparkDf.columns

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 90, Finished, Available, Finished)

In [89]:
dfSplitSparkDftidied = dfSplitSparkDf

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 91, Finished, Available, Finished)

In [92]:
for col in dfSplitSparkDf.columns:

    for pattern in regexp_patterns:

        dfSplitSparkDftidied = dfSplitSparkDftidied.withColumn(col, regexp_replace(col, pattern, ''))

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 94, Finished, Available, Finished)

In [94]:
display(dfSplitSparkDftidied)

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 96, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, aad3a9cc-f16a-44f0-b6c6-a2164a9ef180)

In [95]:
import re

for col in dfSplitSparkDf.columns:

    if re.search('date', col) is True:

        for pattern in regexp_patterns:
            dfSplitSparkDftidied = dfSplitSparkDftidied.withColumn(col, regexp_replace(col, pattern, ''))
            
    if re.search('time', col) is True:

        for pattern in non_temporal_regexp_patterns:
            dfSplitSparkDftidied = dfSplitSparkDftidied.withColumn(col, regexp_replace(col, pattern, ''))

    else:
        for pattern in non_temporal_regexp_patterns:
            dfSplitSparkDftidied = dfSplitSparkDftidied.withColumn(col, regexp_replace(col, pattern, ''))

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 97, Finished, Available, Finished)

In [96]:
display(dfSplitSparkDftidied)

StatementMeta(, 188a2a3f-45e7-46b9-988d-2932e13b39c7, 98, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3b05837b-14ba-41c1-b6ba-962ba005fa75)